In [4]:
import pandas as pd
import numpy as np

%run Util.ipynb
%run Constant.ipynb
%run Policy_Loader.ipynb
%run BreakSectionManager.ipynb
%run Operation.ipynb
%run Evaluation.ipynb


# 沿着时间线，模拟买卖
class ProRocketRegression:
    
    def __init__(self,scale,start_date,end_date,policy_name):
        self.scale = scale
        self.start_date = start_date
        self.end_date = end_date
        self.policy_name = policy_name
        
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        self.account = Account(self.context,self.policy_name)
        
        self.operations={}
        
        
    def get_to_buy_candidate_stocks(self,cur_date,all_sections):
        already_bought = list(self.account.position.keys())
        
        part = all_sections[all_sections['d_s_date']==cur_date]
        
        if(part.shape[0]>0):
            stocks = part['symbol'].values
            
            for s in stocks:
                already_bought.append(s)
    
            if(self.debug==True):
                print(cur_date+' found to buy stocks '+ str(already_bought))
                
        return already_bought
    
    
    def get_to_sell_candidate_stocks(self,cur_date):
        symbols = list(self.account.position.keys())
        
        return symbols
        
    def get_all_sections(self):
        symbols = get_symbols(self.scale,home)
        
        all_sections = pd.DataFrame(columns=break_section_columns)
        for symbol in symbols:
            sectionManager = BreakSectionManager(symbol,self.context)
            section_df = sectionManager.get_all_break_sections()
            
            all_sections = pd.concat([section_df,all_sections], ignore_index=True, sort=False)
            
        return all_sections
    
    def get_operation(self,symbol):
        if(symbol in self.operations):
            return self.operations[symbol]
        else:
            operation = Operation(self.account,symbol,self.context,self.policy_name)
            self.operations[symbol] = operation
            
            return operation
        
    
    def try_to_buy(self,cur_date,candidates):
        if(candidates is None):
            return
        
        for symbol in candidates:
            operation = self.get_operation(symbol)
        
            if(operation.is_buy_point(cur_date)==True):
                operation.open_opsition(cur_date)


    def try_to_sell(self,cur_date,candidates):
        for symbol in candidates:
            operation = self.get_operation(symbol)
            
            if(operation.is_sell_point(cur_date)==True):
                if(self.debug==True):
                    print('try to sell '+ symbol)

                operation.sell_stock(cur_date)
            
    
#     def persist(self):
#         self.account.get_asset_df().to_csv(self.build_asset_file_path(symbol,regression_end_date),index=False)
#         self.account.deal_df.to_csv(self.build_deal_file_path(symbol,regression_end_date),index=False)
        
    def draw_asset(self,account,operation_df,symbol,regression_end_date):
        if(self.policy_loader.enable_painter()==True):
            painter = AssetPainter()

            asset_file = self.build_asset_file_path(symbol,regression_end_date)
            painter.draw_month_profit_bars(asset_file)
            painter.draw_audit_asset_bars(asset_file)

            operation_file=self.build_operation_file_path(symbol,regression_end_date)
            painter.draw_operation_asset_bars(operation_file)
        
    def start_regression(self):
        start_datetime = to_datetime(self.start_date)
        end_datetime = to_datetime(self.end_date)
        
        all_sections = self.get_all_sections()
        market = Metric('SH#999999',self.context)
        
        if(self.debug==True):
            print('regression start ... ')
            
        while(start_datetime<=end_datetime):
            cur_date = start_datetime.strftime(YMD_format)
            
            if(market.is_today_open(cur_date)):
                self.account.daily_audit(cur_date)

                if(self.account.can_open_new_stock()==True):
                    candidates = self.get_to_buy_candidate_stocks(cur_date,all_sections)
                    self.try_to_buy(cur_date,candidates)
                    
                if(self.account.has_shares()==True):
                    candidates = self.get_to_sell_candidate_stocks(cur_date)
                    self.try_to_sell(cur_date,candidates)
                    
            start_datetime += dt.timedelta(days = 1)
  
        evaluation = Evaluation(self.account,self.context,self.policy_name)
        evaluation_df = evaluation.get_evaluation_report_of_multip_stocks(self.start_date,self.end_date)

        if(self.debug==True):
            print('regression done !')
            
        return evaluation_df



In [5]:
scale = 1
start = '2000-02-01'
end = '2019-08-30'
policy_name='rocket_break_v6_enable_multip_stocks.json'

regression = ProRocketRegression(scale,start,end,policy_name)
evaluation_df = regression.start_regression()

evaluation_df.T

regression start ... 
2006-04-11 found to buy stocks ['SZ#000573']
to buy shares:15360.9831029 total_asset:100000.0 price_gap:0.1302
max_cost:100000.0 to_buy_percent:2 to_buy_stock_money:2000.0
SZ#000573 cur open deals


,symbol,date,price,shares,cost,increase_per,type,reason,stock_space,account_space,status
1,SZ#000573,2006-04-11,1.86,1000,1860.0,0,Buy,break_highest,1.83,1.83,open


2006-05-15 triggered add position
to buy shares:13327.5747508 total_asset:100290.0 price_gap:0.1505
max_cost:100290.0 to_buy_percent:60 to_buy_stock_money:60174.0
2006-05-15 triggered add position
SZ#000573 cur open deals


,symbol,date,price,shares,cost,increase_per,type,reason,stock_space,account_space,status
1,SZ#000573,2006-04-11,1.86,1000,1860.0,0,Buy,break_highest,1.83,1.83,open
2,SZ#000573,2006-05-15,2.15,27900,59985.0,15.5914,Buy,buy_reason_increase_per,38.77,38.77,open


2006-06-16 triggered add position
to buy shares:12450.0 total_asset:113295.0 price_gap:0.182
max_cost:113295.0 to_buy_percent:38 to_buy_stock_money:43052.1
2006-06-16 triggered add position
SZ#000573 cur open deals


...

,symbol,date,price,shares,cost,increase_per,type,reason,stock_space,account_space,status
1,SZ#000573,2006-04-11,1.8600,1000,1860.00,0,Buy,break_highest,1.83,1.83,done
2,SZ#000573,2006-05-15,2.1500,27900,59985.00,15.5914,Buy,buy_reason_increase_per,38.77,38.77,done
3,SZ#000573,2006-06-16,2.6000,14600,37960.00,39.7849,Buy,buy_reason_increase_per,74.77,74.77,done
4,SZ#000573,2006-07-31,2.3300,43500,101355.00,25.2688,Sell,stop_profit,0.00,0.00,done
5,SZ#000573,2006-12-04,2.7300,700,1911.00,0,Buy,break_highest,1.85,1.85,done
6,SZ#000573,2006-12-08,2.5389,700,1777.23,-7.69231,Sell,stop_loss,0.00,0.00,done
7,SZ#000573,2009-06-10,4.3400,400,1736.00,0,Buy,break_highest,1.68,1.68,done
8,SZ#000573,2009-07-01,4.8400,12500,60500.00,11.5207,Buy,buy_reason_increase_per,38.51,38.51,done
9,SZ#000573,2009-07-27,5.2800,7400,39072.00,21.659,Buy,buy_reason_increase_per,73.23,73.23,done
10,SZ#000573,2009-08-07,4.7600,8100,38556.00,9.67742,Sell,reducing_position,99.81,99.81,done


try to sell SZ#000573
sell_type stop_loss
2017-01-04 sell SZ#000573 price:9.9696 100 position:{}
SZ#000573 cur open deals


,symbol,date,price,shares,cost,increase_per,type,reason,stock_space,account_space,status
1,SZ#000573,2006-04-11,1.8600,1000,1860.00,0,Buy,break_highest,1.83,1.83,done
2,SZ#000573,2006-05-15,2.1500,27900,59985.00,15.5914,Buy,buy_reason_increase_per,38.77,38.77,done
3,SZ#000573,2006-06-16,2.6000,14600,37960.00,39.7849,Buy,buy_reason_increase_per,74.77,74.77,done
4,SZ#000573,2006-07-31,2.3300,43500,101355.00,25.2688,Sell,stop_profit,0.00,0.00,done
5,SZ#000573,2006-12-04,2.7300,700,1911.00,0,Buy,break_highest,1.85,1.85,done
6,SZ#000573,2006-12-08,2.5389,700,1777.23,-7.69231,Sell,stop_loss,0.00,0.00,done
7,SZ#000573,2009-06-10,4.3400,400,1736.00,0,Buy,break_highest,1.68,1.68,done
8,SZ#000573,2009-07-01,4.8400,12500,60500.00,11.5207,Buy,buy_reason_increase_per,38.51,38.51,done
9,SZ#000573,2009-07-27,5.2800,7400,39072.00,21.659,Buy,buy_reason_increase_per,73.23,73.23,done
10,SZ#000573,2009-08-07,4.7600,8100,38556.00,9.67742,Sell,reducing_position,99.81,99.81,done


operation_log
['SZ#000573', '2016-12-16', '2017-01-04', 10.720000000000001, 9.9695999999999998, 100, 0.0, -7.0, 87103.400000000009, 'unknown', 24.85]

year_returns
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4, 0.0, 0.0, -14.83, 0.0, -0.12, 0.0, -0.25, -0.13, 7.16, -8.6, -0.06, 0.0, 0.0]
Compound avg year rate
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.2, 0.2, -1.2, -1.1, -1.0, -1.0, -0.9, -0.9, -0.3, -0.8, -0.8, -0.7]
regression done !


,1
symbol,all_symbols
start_date,2000-02-01
end_date,2019-08-30
policy_name,rocket_break_v6_enable_multip_stocks.json
short,30
mid,240
in_hand_asset_drawdown,14.23
avg_drawdown,14.1891
loss_times,0
float_profit_drawdown,24.85


In [ ]:
regression.account.asset.plot()